# Working with TabularDataResource

## Adding packages for ChickenScratch and Sql Server

In [3]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"
#r "nuget: ChickenScratch.Interactive"
#r "nuget: Fsharp.Core"

open ChickenScratch

Installed Packages ChickenScratch.Interactive, 1.2.0 Fsharp.Core, 7.0.200 Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.23205.1

Loading extensions from `C:\Users\dave\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.23205.1\interactive-extensions\dotnet\Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

Loading extension script from `C:\Users\dave\.nuget\packages\chickenscratch.interactive\1.2.0\interactive-extensions\dotnet\extension.dib`

## Connecting to the database and running a simple query

This sample makes use of the [AdventureWorks Sample Database](https://learn.microsoft.com/en-us/sql/samples/adventureworks-install-configure?view=sql-server-ver16&tabs=ssms).

In [4]:
#!connect mssql --kernel-name advWorks "Data Source=(localdb)\Sql2022Express;Initial Catalog=AdventureWorksLT2019;Integrated Security=True"

Kernel added: #!sql-advWorks

In [9]:
#!sql-advWorks --name dbProducts

select top(5) 
    prod.ProductId, 
    prod.Name as ProductName, 
    prod.Color, 
    prod.ListPrice, 
    prod.Weight, 
    prod.ProductCategoryID, 
    prod.SellStartDate, 
    cat.Name as CategoryName
from SalesLT.Product prod
    join SalesLT.ProductCategory cat on prod.ProductCategoryID = cat.ProductCategoryID



(5 rows affected)

ProductId,ProductName,Color,ListPrice,Weight,ProductCategoryID,SellStartDate,CategoryName
680,"HL Road Frame - Black, 58",Black,1431.50,1016.04,18,2002-06-01 00:00:00Z,Road Frames
706,"HL Road Frame - Red, 58",Red,1431.50,1016.04,18,2002-06-01 00:00:00Z,Road Frames
707,"Sport-100 Helmet, Red",Red,34.99,<null>,35,2005-07-01 00:00:00Z,Helmets
708,"Sport-100 Helmet, Black",Black,34.99,<null>,35,2005-07-01 00:00:00Z,Helmets
709,"Mountain Bike Socks, M",White,9.50,<null>,27,2005-07-01 00:00:00Z,Socks


## Setting up some types to work with

In [11]:
type Color = Black | Red | White

type Category = 
| RoadFrames = 18
| Helmets = 35
| Socks = 27

type Product = {
    Id : int
    Name : string
    Color : Color
    ListPrice : decimal
    Weight : decimal option
    SellStartDate : DateTime
    Category : Category
}

## Getting strongly-typed data from the TabularDataResource using ChickenScratch

In [13]:
#!share dbProducts --from sql-advWorks --as productsData

let products = 
    productsData[0].Data
    |> Seq.map (fun row ->
            { 
                Id = row |> TabDataRow.Value "ProductId"
                Name = row |> TabDataRow.Value "ProductName"
                Color = row |> TabDataRow.UnionValue "Color"
                ListPrice = row |> TabDataRow.Value "ListPrice"
                Weight = row |> TabDataRow.OptionalValue "Weight"
                SellStartDate = row |> TabDataRow.Value "SellStartDate"
                Category = row |> TabDataRow.EnumValue "CategoryName"
            }
        )
    |> Seq.toList

products

index value 0 { Id = 680\n Name = "HL Road Frame - Black, 58"\n Color = Black\n ListPrice = 1431.50M\n Weight = Some 1016.04M\n SellStartDate = 6/1/2002 12:00:00 AM\n Category = RoadFrames } Id 680 Name HL Road Frame - Black, 58 Color Black ListPrice 1431.50 Weight Some(1016.04) Value 1016.04 SellStartDate 2002-06-01 00:00:00Z Category RoadFrames 1 { Id = 706\n Name = "HL Road Frame - Red, 58"\n Color = Red\n ListPrice = 1431.50M\n Weight = Some 1016.04M\n SellStartDate = 6/1/2002 12:00:00 AM\n Category = RoadFrames } Id 706 Name HL Road Frame - Red, 58 Color Red ListPrice 1431.50 Weight Some(1016.04) Value 1016.04 SellStartDate 2002-06-01 00:00:00Z Category RoadFrames 2 { Id = 707\n Name = "Sport-100 Helmet, Red"\n Color = Red\n ListPrice = 34.99M\n Weight = None\n SellStartDate = 7/1/2005 12:00:00 AM\n Category = Helmets } Id 707 Name Sport-100 Helmet, Red Color Red ListPrice 34.99 Weight <null> SellStartDate 2005-07-01 00:00:00Z Category Helmets 3 { Id = 708\n Name = "Sport-100 Helmet, Black"\n Color = Black\n ListPrice = 34.99M\n Weight = None\n SellStartDate = 7/1/2005 12:00:00 AM\n Category = Helmets } Id 708 Name Sport-100 Helmet, Black Color Black ListPrice 34.99 Weight <null> SellStartDate 2005-07-01 00:00:00Z Category Helmets 4 { Id = 709\n Name = "Mountain Bike Socks, M"\n Color = White\n ListPrice = 9.50M\n Weight = None\n SellStartDate = 7/1/2005 12:00:00 AM\n Category = Socks } Id 709 Name Mountain Bike Socks, M Color White ListPrice 9.50 Weight <null> SellStartDate 2005-07-01 00:00:00Z Category Socks

Any value can be interpreted as optional, which will cause `null` to be converted to `None`.

In [16]:
printfn "%A, %A"
    (productsData[0].Data |> Seq.head |> TabDataRow.OptionalValue<int> "ProductId")
    (productsData[0].Data |> Seq.head |> TabDataRow.OptionalUnionValue<Color> "Color")


Some 680, Some Black


Enums can be parsed from strings, or converted from the underlying type.

In [19]:
printfn "%A, %A"
    (productsData[0].Data |> Seq.head |> TabDataRow.Value<Category> "ProductCategoryID")  
    (productsData[0].Data |> Seq.head |> TabDataRow.EnumValue<Category> "CategoryName")

RoadFrames, RoadFrames


Union types must be pascal case, but Enum types may have a different convention.  If needed, you can specify the naming convention for your enum types.  Naming conventions are found in the `ChickenScratch.Utility` namespace.

In [21]:
open ChickenScratch.Utility

type SnakeCategory =
| road_frames = 18
| helmets = 35
| socks = 27

type CamelCategory =
| roadFrames = 18
| helmets = 35
| socks = 27

printfn "%A, %A"
    (productsData[0].Data |> Seq.head |> TabDataRow.EnumValueWithConvention<SnakeCategory> SnakeCase "CategoryName")
    (productsData[0].Data |> Seq.head |> TabDataRow.EnumValueWithConvention<CamelCategory> CamelCase "CategoryName")

road_frames, roadFrames
